In [6]:
import psycopg2

In [1]:
from os.path import expanduser
import json

class Credentials:

    def __init__(self, auth_filepath=expanduser("~/.credentials.json")):

        self.auth_filepath = auth_filepath

        with open(auth_filepath) as fin:
            self.json = json.load(fin)

    def get(self, parameter, default=None):
        return self.json.get(parameter, default)

class DatabaseDetails:

    def __init__(self, username, password, database, host, port):
        self.username = username
        self.password = password
        self.database = database
        self.host = host
        self.port = port

    def get_credentials(self):
        if self.username and self.password:
            return {
                'computron_username': self.username,
                'computron_password': self.password
            }


In [2]:
DEFAULT_DB = 'data_depot'
DEFAULT_HOST = 'freshbooks-data.c8exzn6geij3.us-east-1.redshift.amazonaws.com'
DEFAULT_PORT = 5439

class PsycopgConnector:
    '''
    A database connector that uses Psycopg to connect to Redshift.

    How to play:

        from outlier_utils import Credentials()

        creds = Credentials()

        psy_conn = PsycopgConnector(creds)
        df = psy_conn.run_query(sql=sql, return_data=True)

    NOTE: This class commits queries to redshift if return_data=False. This means INSERT, DROP, TRUNCATE, etc. all work against the DB.
    '''

    def __init__(
        self,
        credentials=None,
        username=None,
        password=None,
        db=DEFAULT_DB,
        host=DEFAULT_HOST,
        port=DEFAULT_PORT,
        db_details=None
    ):

        db_details = db_details or DatabaseDetails(username, password, db, host, port)
        credentials = credentials or db_details.get_credentials()

        self.db = db_details.database
        self.host = db_details.host
        self.port = db_details.port

        self.username = credentials.get(
            'computron_username',
            db_details.username
        )
        self.password = credentials.get(
            'computron_password',
            db_details.password
        )

    def _get_connection(self):

        self.conn = psycopg2.connect(
            dbname=self.db,
            user=self.username,
            password=self.password,
            host=self.host,
            port=self.port
        )

        return self.conn

    def run_query(self, sql, return_data=False):

        with self._get_connection() as conn:
            with conn, conn.cursor() as cur:
                if return_data:
                    return pd.read_sql(sql=sql, con=conn)
                else:
                    cur.execute(sql)
        conn.close()


In [3]:
creds = Credentials()

In [4]:
pig = PsycopgConnector(creds)

In [7]:
sql = """
    SELECT
    * 
    FROM report_systems
    LIMIT 10
"""

pig.run_query(sql, return_data=True)


OperationalError: FATAL:  password authentication failed for user "dwahid"
FATAL:  password authentication failed for user "dwahid"
